# Aries Basic Controller - Openmined PryVote Voter

This tutorial runs through the issuer-api from the perspective of a holder. It should be run alongside the [openmind-pryvote-community issuer notebook](http://127.0.0.1:8888/notebooks/openmind-pryvote-community.ipynb) from Openmined PryVote Community (Alice)'s perspective.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8092
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8091"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

## 2. Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

## 8. Check Credential Exchange Records

The agent will have at least one record if you have run through the [openmind-pryvote-community issuer notebook](http://127.0.0.1:8888/notebooks/openmind-pryvote-community.ipynb) notebook up until send credential.

In [5]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange 24f846b8-13db-40da-93c8-a3502e1db0ad, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## 10. Request Credential from Openmined PryVote Community (Red) Issuer

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [6]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 24f846b8-13db-40da-93c8-a3502e1db0ad, role: holder, state: request_sent
Handle Credentials
Credential exchange 24f846b8-13db-40da-93c8-a3502e1db0ad, role: holder, state: request_sent
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange 24f846b8-13db-40da-93c8-a3502e1db0ad, role: holder, state: credential_received
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## 11. Store the credential

Once the Openmined PryVote Community (Alice) issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [8]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 24f846b8-13db-40da-93c8-a3502e1db0ad, role: holder, state: credential_received


In [9]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 24f846b8-13db-40da-93c8-a3502e1db0ad, role: holder, state: credential_acked
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Credential exchange 24f846b8-13db-40da-93c8-a3502e1db0ad, role: holder, state: credential_acked


# 13. Get Proof Request Records

In [10]:
response = await agent_controller.proofs.get_records()
print(response)

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

# print('Presentation Exchange ID\n')
# print(response['results'][0]['presentation_exchange_id'])
# pres_x_id = response['results'][0]['presentation_exchange_id']
# print('Presentation Request Object\n')
# print(response['results'][0]['presentation_request'])
# print('Requested Attributes\n')
# print(response['results'][0]['presentation_request']['requested_attributes'])
# requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
# print('Requested Predicates\n')
# print(response['results'][0]['presentation_request']['requested_predicates'])
# requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'results': [{'updated_at': '2020-09-09 18:45:31.255941Z', 'state': 'request_received', 'role': 'prover', 'connection_id': 'a775ff83-a86c-44cb-b78e-ad161d2fcf9d', 'presentation_request': {'name': 'Proof of Openmined Community Member', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_community_vote_uuid': {'name': 'community_vote'}}, 'requested_predicates': {'0_community_member_GE_uuid': {'name': 'community_member', 'p_type': '>=', 'p_value': 1, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '756179962795484379982934'}, 'presentation_exchange_id': 'bd89a182-9bc0-407c-b126-1dee741c52d5', 'trace': False, 'thread_id': '0e9dffa6-0628-4bbb-9ee0-43c7975900ec', 'initiator': 'external', 'created_at': '2020-09-09 18:45:31.255941Z'}, {'updated_at': '2020-09-09 18:45:43.835695Z', 'state': 'request_received', 'role': 'prover', 'connection_id': 'a775ff83-a86c-44cb-b78e-ad161d2fcf9d', 'pr

# 14. Fetch Credentials for Proof Presentation

In [11]:
# presentation_exchange_id = '0b4e53ab-f1ba-4b7d-bd0f-bdab3a027626'
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "True"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
request = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(request)
print("\n")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'My OM Credential', 'attrs': {'name': 'Bob', 'community_member': '1'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_community_member_GE_uuid', '0_name_uuid']}]

Generate the proof
{'requested_predicates': {'0_community_member_GE_uuid': {'cred_id': 'My OM Credential'}}, 'requested_attributes': {'0_name_uuid': {'cred_id': 'My OM Credential', 'revealed': True}}, 'self_attested_attributes': {'0_community_vote_uuid': 'True'}}


{'0_community_member_GE_uuid': {'cred_id': 'My OM Credential'}}
{'0_name_uuid': {'cred_id': 'My OM Credential', 'revealed': True}}
{'0_community_vote_uuid': 'True'}



# 15. Send a Proof Presentation

In [14]:
proof_body = {
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
    "requested_predicates": predicates,
    "trace": False
}

print("\nSend the proof to Openmined Community Agent (Alice)")
send_presentation = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof_body)
print(response)


Send the proof to Openmined Community Agent (Alice)


ClientResponseError: 500, message='Internal Server Error', url=URL('http://bob-agent:8051/present-proof/records/bd89a182-9bc0-407c-b126-1dee741c52d5/send-presentation')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [15]:
response = await agent_controller.terminate()
print(response)

None
